In [ ]:
!pip install youtube-transcript-api
!pip install googletrans==4.0.0-rc1
!pip install aiohttp

# Transcribe (Get the transcript)

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

def get_youtube_transcript(video_id, lang='hi'):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[lang])
        return transcript
    except Exception as e:
        print(f"Error retrieving transcript: {e}")
        return None

# Example usage
video_id = ''
transcript = get_youtube_transcript(video_id)

# Translate using Google API

In [ ]:
import aiohttp
import asyncio
import nest_asyncio

nest_asyncio.apply()

async def translate_text(session, text, target_language='en'):
    url = 'https://translate.googleapis.com/translate_a/single'
    params = {
        'client': 'gtx',
        'sl': 'hi',
        'tl': target_language,
        'dt': 't',
        'q': text
    }
    async with session.get(url, params=params) as response:
        result = await response.json()
        return result[0][0][0]

async def batch_translate(captions):
    async with aiohttp.ClientSession() as session:
        tasks = [translate_text(session, entry['text']) for entry in captions]
        translations = await asyncio.gather(*tasks)
        return [{'start': captions[i]['start'], 'text': translations[i]} for i in range(len(captions))]

# Usage
def run_translation(captions):
    return asyncio.run(batch_translate(captions))

if 'transcript' in globals():
    translated_captions = run_translation(transcript)


# Save as SRT

In [ ]:
def save_as_srt(captions, filename='translated_captions.srt'):
    with open(filename, 'w', encoding='utf-8') as f:
        for idx, entry in enumerate(captions):
            start_time = entry['start']
            end_time = entry['start'] + 5
            start_str = convert_to_srt_timestamp(start_time)
            end_str = convert_to_srt_timestamp(end_time)

            f.write(f"{idx+1}\n")
            f.write(f"{start_str} --> {end_str}\n")
            f.write(f"{entry['text']}\n\n")

def convert_to_srt_timestamp(seconds):
    hrs, secs = divmod(seconds, 3600)
    mins, secs = divmod(secs, 60)
    millisecs = int((secs - int(secs)) * 1000)
    return f"{int(hrs):02}:{int(mins):02}:{int(secs):02},{millisecs:03}"

if translated_captions:
    save_as_srt(translated_captions)
